<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/nafnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
!nvidia-smi
%cd /content

!git clone https://github.com/megvii-research/NAFNet

%cd /content/NAFNet

!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext
!pip install Pillow
!apt-get install -y ffmpeg
!pip install moviepy imageio imageio-ffmpeg --quiet

%cd /content/NAFNet

import gdown

import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
from google.colab import files
import shutil
import glob

from google.colab import files
import shutil
from google.colab import files
from PIL import Image
import numpy as np

from tqdm import tqdm
import sys
import re
from PIL import Image, ImageEnhance

import PIL.Image
import numpy as np
from IPython.display import display
from IPython.display import display, clear_output

import ipywidgets as widgets
from IPython.display import display

%cd /content/NAFNet

# Deblur
if not os.path.exists("./experiments/pretrained_models/NAFNet-REDS-width64.pth"):
  gdown.download('https://drive.google.com/uc?id=14D4V4raNYIOhETfcuuLI3bGLB-OYIv6X', "./experiments/pretrained_models/", quiet=False)

def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)

def sr_display(LR_l, LR_r, SR_l, SR_r):
  h,w = SR_l.shape[:2]
  LR_l = cv2.resize(LR_l, (w,h), interpolation=cv2.INTER_CUBIC)
  LR_r = cv2.resize(LR_r, (w,h), interpolation=cv2.INTER_CUBIC)
  fig = plt.figure(figsize=(w//40, h//40))
  ax1 = fig.add_subplot(2, 2, 1)
  plt.title('Input image (Left)', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(2, 2, 2)
  plt.title('NAFSSR output (Left)', fontsize=16)
  ax2.axis('off')
  ax1.imshow(LR_l)
  ax2.imshow(SR_l)

  ax3 = fig.add_subplot(2, 2, 3)
  plt.title('Input image (Right)', fontsize=16)
  ax3.axis('off')
  ax4 = fig.add_subplot(2, 2, 4)
  plt.title('NAFSSR output (Right)', fontsize=16)
  ax4.axis('off')
  ax3.imshow(LR_r)
  ax4.imshow(SR_r)

  plt.subplots_adjust(wspace=0.04, hspace=0.04)

def stereo_image_inference(model, img_l, img_r, save_path):
      img = torch.cat([img_l, img_r], dim=0)
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      img_L = visuals['result'][:,:3]
      img_R = visuals['result'][:,3:]
      img_L, img_R = tensor2img([img_L, img_R])

      imwrite(img_L, save_path.format('L'))
      imwrite(img_R, save_path.format('R'))

%cd /content/NAFNet

opt_path = 'options/test/REDS/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)
clear_output(wait=True)

In [ ]:
#@title ##**Google drive** { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ##**Upload** { display-mode: "form" }
video_source = "google drive" #@param ["pc", "google drive"]

if video_source == "pc":
    from google.colab import files
    uploaded = files.upload()
    # Get the name of uploaded file
    file_name = list(uploaded.keys())[0]
    print(f"Selected file: {file_name}")
elif video_source == "google drive":
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    import ipywidgets as widgets
    from IPython.display import display

    # Search for video files in Google Drive
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv')

    video_files = []
    for root, dirs, files in os.walk('/content/drive/MyDrive'):
        for file in files:
            if file.lower().endswith(video_extensions):
                file_path = os.path.join(root, file)
                video_files.append(file_path)

    if not video_files:
        print("No video files found in Google Drive.")
    else:
        # Create dropdown list for file selection
        video_dropdown = widgets.Dropdown(
            options=video_files,
            description='Select video file:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='90%')
        )

        # Button for confirming selection
        select_button = widgets.Button(description='Select file')
        output = widgets.Output()

        def on_button_clicked(b):
            with output:
                global file_name
                file_name = video_dropdown.value
                print(f"Selected file: {file_name}")

        select_button.on_click(on_button_clicked)

        display(video_dropdown, select_button, output)

In [ ]:
#@title ##**Config** { display-mode: "form" }

continue_status = "new" #@param ["continue","new"]
opacity = "20" #@param [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

sharpness_and_upscale = "yes" #@param ["yes","no"]
sharpness_factor = 3  # @param {type:"slider", min:0.0, max:7.0, step:0.1}
upscale_option = "height 720px"  # @param ["no", "height 720px", "height 1080px", "height 2160px", "2x", "4x"]

from PIL import ImageEnhance

%cd /content/NAFNet

path = '/content/drive/MyDrive/'
deblur_upload_folder = '/content/drive/MyDrive/deblur_input'
deblur_result_folder = '/content/drive/MyDrive/deblur_output'
deblur_result_folder_overlay = '/content/drive/MyDrive/deblur_output_overlay'
deblur_result_folder_sharp = '/content/drive/MyDrive/deblur_output_overlay_sharp'

base_result_folder = deblur_result_folder

if not os.path.exists(deblur_upload_folder):
    os.makedirs(deblur_upload_folder)

if not os.path.exists(deblur_result_folder):
    os.makedirs(deblur_result_folder)

if not os.path.exists(deblur_result_folder_overlay):
    os.makedirs(deblur_result_folder_overlay)

if not os.path.exists(deblur_result_folder_sharp):
    os.makedirs(deblur_result_folder_sharp)

if continue_status == "new":
  if os.path.isdir(deblur_upload_folder):
      shutil.rmtree(deblur_upload_folder)

  os.makedirs(deblur_upload_folder)

  if os.path.isdir(deblur_result_folder):
      shutil.rmtree(deblur_result_folder)

  os.makedirs(deblur_result_folder)

  if os.path.isdir(deblur_result_folder_overlay):
      shutil.rmtree(deblur_result_folder_overlay)

  os.makedirs(deblur_result_folder_overlay)

  if os.path.isdir(deblur_result_folder_sharp):
      shutil.rmtree(deblur_result_folder_sharp)

  os.makedirs(deblur_result_folder_sharp)


In [ ]:
#@title ##**Sequence** { display-mode: "form" }
import cv2
import imageio
import os
import tqdm
import subprocess
import numpy as np
import time

video_convertion = "no" #@param ["yes","no"]
if (video_convertion == "yes"):

  output_file = os.path.splitext(file_name)[0] + "_converted.mp4"

  !ffmpeg -i "{file_name}" -c:v libx264 -preset slow -crf 18 -c:a aac -b:a 192k -strict experimental -y "{output_file}"

  if os.path.exists(output_file):
      file_name = output_file
      print(f"Video successfully converted: {file_name}")
  else:
      print("Video conversion error")

library = "imageio" #@param ["cv2","pyav","imageio","ffmpeg","skvideo","scipy","moviepy"]
delay = "0.05" #@param [0, 0.01, 0.05, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

full_path = os.path.join(path, file_name)

if (library == "cv2"):
    cmd = 'ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 ' + full_path
    output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

    cmd = 'ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of csv=p=0 ' + full_path
    output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()
    vidcap = cv2.VideoCapture(full_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    vidcap = cv2.VideoCapture(full_path)
    success, image = vidcap.read()

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_cv2 = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    while success:
        try:
            cv2.imwrite(f"{deblur_upload_folder}/{int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):09d}.jpg", image)
        except cv2.error:
            print("Error writing to disk. Retrying...")
            continue
        pbar_cv2.update(1)
        time.sleep(float(delay))
        success, image = vidcap.read()

    pbar_cv2.close()
    vidcap.release()

elif (library == "pyav"):
    !pip install av
    import av
    video = av.open(full_path)

    fps_value = video.streams.video[0].average_rate
    frame_count = video.streams.video[0].frames

    if fps_value is not None:
        duration = frame_count / fps_value

        print("FPS: ", fps_value)
        print("Duration: ", duration)
        print("Frames: ", frame_count)

        tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
        pbar_pyav = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
        for packet in video.demux():
            for frame in packet.decode():
                img = frame.to_ndarray(format='rgb24')
                img = img.astype('uint8')
                img_path = f"{deblur_upload_folder}/{packet.pts:09d}.jpg"
                imageio.imwrite(img_path, img)
                pbar_pyav.update(1)
                time.sleep(float(delay))

        pbar_pyav.close()
        video.close()
    else:
        print("Error: FPS value is None.")

elif (library == "imageio"):
    video = imageio.get_reader(full_path)

    fps = video.get_meta_data()['fps']
    frame_count = video.count_frames()
    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_imageio = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video):
      img_path = f"{deblur_upload_folder}/{i+1:09d}.jpg"
      if os.path.isfile(img_path):
          pbar_imageio.update(1)
          continue
      while True:
          try:
              imageio.imwrite(img_path, frame)
          except Exception as e:
              print(f"Error writing to disk: {str(e)}. Retrying...")
              continue
          break
      pbar_imageio.update(1)
      time.sleep(float(delay))

    pbar_imageio.close()
    video.close()

elif (library == "ffmpeg"):
    !pip install ffmpeg-python
    import ffmpeg

    probe = ffmpeg.probe(full_path)
    video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
    fps = video_info['r_frame_rate']
    duration = float(video_info['duration'])
    frame_count = int(video_info['nb_frames'])

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    pbar_ffmpeg = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    process = (
        ffmpeg
        .input(full_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', qscale=0)
        .run_async(pipe_stdout=True)
    )

    for i in range(frame_count):
        try:
            raw_video = process.stdout.read(video_info['width'] * video_info['height'] * 3)
            frame = np.frombuffer(raw_video, dtype='uint8').reshape((video_info['height'], video_info['width'], 3))
            frame_path = f"{deblur_upload_folder}/{i:09d}.jpg"
            if os.path.isfile(frame_path):
              pbar_ffmpeg.update(1)
              continue
            imageio.imwrite(frame_path, frame)
        except Exception as e:
            print(f"Error writing to disk: {str(e)}. Retrying...")
            continue
        pbar_ffmpeg.update(1)
        time.sleep(float(delay))

    pbar_ffmpeg.close()
    process.wait()

elif (library == "skvideo"):
    !pip install scikit-video
    import skvideo.io

    video = skvideo.io.vread(full_path)

    fps_str = skvideo.io.ffprobe(full_path)['video']['@avg_frame_rate']
    fps_parts = fps_str.split('/')
    fps = int(fps_parts[0]) / int(fps_parts[1])
    frame_count = len(video)
    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    outputdict = {'-q:1': '1'}
    pbar_skvideo = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video):
        while True:
            try:
                skvideo.io.vwrite(f"{deblur_upload_folder}/{i:09d}.jpg", frame, outputdict=outputdict)
            except Exception as e:
                print(f"Error writing to disk: {str(e)}. Retrying...")
                continue
            break
        pbar_skvideo.update(1)
        time.sleep(float(delay))

    pbar_skvideo.close()

elif (library == "scipy"):
    import scipy.misc
    import scipy.ndimage

    video = imageio.get_reader(full_path)
    fps_value = video.get_meta_data()['fps']
    frame_count = video.get_length()

    if fps_value is not None:
        duration = frame_count / fps_value

        print("FPS: ", fps_value)

        tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
        pbar_scipy = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
        for i, im in enumerate(video):
            img_path = f"{deblur_upload_folder}/{i+1:09d}.jpg"
            im = Image.fromarray(im)
            im.save(img_path, quality=100)
            pbar_scipy.update(1)
            time.sleep(float(delay))

        pbar_scipy.close()
    else:
        print("Error: FPS value is None.")

elif library == "moviepy":
    from moviepy.editor import VideoFileClip
    from PIL import Image
    video = VideoFileClip(full_path)
    fps_value = video.fps
    frame_count = int(video.duration * fps_value)

    print("FPS: ", fps_value)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_moviepy = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video.iter_frames()):
        img_path = f"{deblur_upload_folder}/{i+1:09d}.jpg"
        im = Image.fromarray(frame)
        im.save(img_path, quality=100)
        pbar_moviepy.update(1)
        time.sleep(float(delay))

    pbar_moviepy.close()

In [ ]:
#@title ##**Check for missing frames(optional)** { display-mode: "form" }
import os
import time

frame_dir = '/content/drive/MyDrive/deblur_input'
frames = None
while frames is None:
    try:
        frames = [int(f.split('.')[0]) for f in os.listdir(frame_dir) if f.endswith('.jpg')]
    except:
        print("Error reading the list of files. Retrying in 2 seconds")
        time.sleep(2)

min_frame = min(frames)
max_frame = max(frames)
print(min_frame)
print(max_frame)


In [ ]:
#@title ##**Run deblur** { display-mode: "form" }
%cd /content/NAFNet
import glob
import os
import time
from tqdm import tqdm


upload_files = set()
while not upload_files:
    try:
        upload_files = set(os.listdir(deblur_upload_folder))
    except OSError as e:
        print(f"Error: {e}, retrying...")
        time.sleep(2)

result_files = set(os.listdir(deblur_result_folder))

input_list = [file_path for file_path in glob.glob(os.path.join(deblur_upload_folder, '*'))
              if os.path.basename(file_path) not in result_files and os.path.basename(file_path) in upload_files]


for input_path in tqdm(input_list, desc="Processing images"):
    img_input = imread(input_path)
    inp = img2tensor(img_input)
    output_path = os.path.join(deblur_result_folder, os.path.basename(input_path))
    single_image_inference(NAFNet, inp, output_path)


#check missing frames
frames = None
while frames is None:
    try:
        frames = [int(f.split('.')[0]) for f in os.listdir(deblur_result_folder) if f.endswith('.jpg')]
    except:
        print("Error reading the list of files. Retrying in 2 seconds")
        time.sleep(2)

min_frame = min(frames)
max_frame = max(frames)
print(min_frame)
print(max_frame)

missing_frames = []
for i in range(min_frame, max_frame + 1):
    if i not in frames:
        missing_frames.append(i)

if len(missing_frames) > 0:
    print(f"Missing frames: {missing_frames}")
else:
    print("All frames present")


In [ ]:
#@title ##**Run overlay** { display-mode: "form" }
if opacity != "100":
  base_result_folder = deblur_result_folder_overlay

  if os.path.isdir(deblur_result_folder_overlay):
      shutil.rmtree(deblur_result_folder_overlay)
  os.makedirs(deblur_result_folder_overlay)

  max_retries = 10
  retries = 0
  while retries < max_retries:
      try:
          filenames = os.listdir(deblur_upload_folder)
          break
      except:
          print("Error getting file list. Trying", retries+1, "from", max_retries)
          retries += 1
          time.sleep(2)

  for filename in filenames:

      try:
          img1 = Image.open(os.path.join(deblur_upload_folder, filename))
          img2 = Image.open(os.path.join(deblur_result_folder, filename))
      except:
          print(f"Error opening file {filename}")
          time.sleep(2)
          continue


      img1 = img1.convert("RGBA")
      img2 = img2.convert("RGBA")

      width1, height1 = img1.size
      width2, height2 = img2.size

      img2.putalpha(int(float(opacity) * 2.55))

      if width1 * height1 > width2 * height2:
          img2 = img2.resize((width1, height1))
      else:
          img1 = img1.resize((width2, height2))

      result = Image.alpha_composite(img1, img2)

      result = result.convert("RGB")

      result.save(os.path.join(deblur_result_folder_overlay, filename), quality=100)
deblur_result_folder = '/content/drive/MyDrive/deblur_output_overlay'

In [ ]:
#@title ##**Run sharpness** { display-mode: "form" }
if sharpness_and_upscale == "yes":
  import shutil
  from tqdm import tqdm
  import os
  import sys
  import re
  from PIL import Image, ImageEnhance

  source_folder = base_result_folder
  destination_folder = deblur_result_folder_sharp

  if not os.path.exists(destination_folder):
      os.makedirs(destination_folder)

  file_list = os.listdir(source_folder)
  file_list.sort()
  frames = [int(f.split('.')[0]) for f in file_list if f.endswith('.jpg')]
  min_frame = min(frames)

  dest_files = os.listdir(destination_folder)
  if dest_files:
      dest_frames = [int(re.findall(r'(\d+)\.jpg', f)[0]) for f in dest_files if re.match(r'\d+\.jpg', f)]
      start_frame = max(dest_frames) + 1
  else:
      start_frame = min_frame

  max_frame = frames[-1]
  print("Max frame:", max_frame)
  files_to_copy = [f"{source_folder}/{frame:09d}.jpg" for frame in range(start_frame, max_frame+1) if f"{frame:09d}.jpg" in file_list]

  total_files = len(files_to_copy)
  batch_size = 10
  num_iterations = (total_files + batch_size - 1) // batch_size

  print("Start frame:", start_frame)
  print("Min frame:", min_frame)
  print("Count files:", total_files)
  print("Count iterations:", num_iterations)

  with tqdm(total=num_iterations) as pbar:
      for i in range(0, total_files, batch_size):
          files_to_copy_batch = files_to_copy[i:i+batch_size]

          for file in files_to_copy_batch:
              filename = os.path.basename(file)
              dest_path = os.path.join(destination_folder, filename)

              original_image = Image.open(file)

              enhancer = ImageEnhance.Sharpness(original_image)
              processed_image = enhancer.enhance(sharpness_factor)

              if upscale_option != "no":
                  width, height = processed_image.size

                  if upscale_option == "height 720px":
                      new_height = 720
                      new_width = int(width * (new_height / height))
                  elif upscale_option == "height 1080px":
                      new_height = 1080
                      new_width = int(width * (new_height / height))
                  elif upscale_option == "height 2160px":
                      new_height = 2160
                      new_width = int(width * (new_height / height))
                  elif upscale_option == "2x":
                      new_width = width * 2
                      new_height = height * 2
                  elif upscale_option == "4x":
                      new_width = width * 4
                      new_height = height * 4
                  else:
                      new_width, new_height = width, height

                  processed_image = processed_image.resize((new_width, new_height), Image.LANCZOS)

              copied = False
              while not copied:
                  try:
                      processed_image.save(dest_path, quality=100)
                      copied = True
                  except:
                      print(f"Failed to save file {dest_path}. Trying again...")
                      continue

          pbar.update(1)

  print("Processing complete")
  base_result_folder = deblur_result_folder_sharp

In [ ]:
#@title ##**Create Video** { display-mode: "form" }

import os
from moviepy.editor import VideoFileClip, ImageSequenceClip

quality = 18  # @param {type:"slider", min:0.0, max:50, step:1}

output_file = os.path.join(path, f"{os.path.splitext(file_name)[0]}_output.mp4")
full_path = os.path.join(path, file_name)

print("Original video:", full_path)

original_video = VideoFileClip(full_path)
fps_of_video = original_video.fps
audio_clip = original_video.audio

print("Frame rate:", fps_of_video)

img_files = os.listdir(base_result_folder)
img_files.sort()

if not img_files:
    print("There are no frames to process in the folder:", destination_folder)
else:
    img_paths = [os.path.join(destination_folder, img_file) for img_file in img_files]

    video_clip = ImageSequenceClip(img_paths, fps=fps_of_video)
    video_clip = video_clip.set_audio(audio_clip)

    video_clip.write_videofile(
        output_file,
        codec="libx264",
        audio_codec="aac",
        ffmpeg_params=["-crf", str(quality)]
    )

    video_clip.close()
    original_video.close()
    del video_clip
    del original_video

    print('Assembly complete. Final video saved to path:', output_file)


In [ ]:
#@title ##**Download file (optional, you can download it faster from Google Drive)** { display-mode: "form" }
from google.colab import files
files.download(output_file)